In [11]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
from cn_general_tool import *

In [12]:
sd, ed = get_sd_ed('2024-11-01', '2024-11-25')

In [13]:
instrument_list = ['AP8888.CZC', 'PR8888.CZC', 'jm8888.DCE']

In [14]:
strategy_sql =  f"""
WITH
data_origin AS (
    SELECT
        date,
        instrument,
        m_avg(close, 5) AS ma_short,
        m_avg(close, 20) AS ma_long,
        IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_long,
        IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_short,
    FROM cn_future_bar1m
)
SELECT
    date,
    instrument,
    signal_long,
    signal_short
FROM data_origin
QUALIFY (signal_long = 1 OR signal_short = 1)
ORDER BY date, instrument
"""

In [15]:
df = dai.query(strategy_sql, filters={"date":[sd, ed], "instrument":instrument_list}).df()
df

,date,instrument,signal_long,signal_short
0,2024-11-01 09:28:00,AP8888.CZC,1,0
1,2024-11-01 09:28:00,PR8888.CZC,0,1
2,2024-11-01 09:31:00,jm8888.DCE,0,1
3,2024-11-01 09:32:00,jm8888.DCE,1,0
4,2024-11-01 09:39:00,PR8888.CZC,1,0
...,...,...,...,...
1006,2024-11-25 22:16:00,jm8888.DCE,1,0
1007,2024-11-25 22:26:00,PR8888.CZC,1,0
1008,2024-11-25 22:46:00,jm8888.DCE,0,1
1009,2024-11-25 22:52:00,jm8888.DCE,1,0


In [16]:
def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    context.subscribe_bar(context.instruments, '1m')

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d %H:%M:%S")]
    
    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction

    instruments_hold  = set(context.get_account_positions().keys())

    for ins in instruments_hold:
        position_long  = context.get_position(ins, Direction.LONG)
        position_short = context.get_position(ins, Direction.SHORT)
        price = data.current(ins,"open")
        if (position_long.current_qty  != 0):
            context.sell_close(ins, position_long.avail_qty, price, order_type=OrderType.MARKET)
        if (position_short.current_qty != 0):
            context.buy_close(ins, position_short.avail_qty, price, order_type=OrderType.MARKET)

    if len(df_now) == 0:
        return

    instruments_long  = set(df_now[df_now["signal_long"]  == 1]["instrument"])
    instruments_short = set(df_now[df_now["signal_short"] == 1]["instrument"])

    for ins in instruments_long:
        price = data.current(ins, "open")
        volume = 1
        context.buy_open(ins, volume, price, order_type=OrderType.MARKET)
    
    for ins in instruments_short:
        price = data.current(ins, "open")
        volume = 1
        context.sell_open(ins, volume, price, order_type=OrderType.MARKET)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""minute""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

BigTrader.raw_perf.read()

[2024-12-03 16:03:30] [info     ] bigtrader.v34 开始运行 ..
[2024-12-03 16:03:30] [info     ] 2024-11-01, 2024-11-25, , future, instruments=3


[2024-12-03 16:03:30] [info     ] bigtrader module V2.2.0
[2024-12-03 16:03:30] [info     ] bigtrader engine v1.10.10 2024-12-02


INFO:MAIN:======== bigtrader pid:6583 version 1.10.10 2024-12-02 ========
INFO:MAIN:bigtrader run_mode:BACKTEST, handle_bar_mode:0, frequency:1m, exchange_mode:BQ2
INFO:MAIN:> process add account:BACKTEST,1,bktfut
INFO:ACCT[bktfut]:AccountEngine: self_calc:1, validate_self_trading:0, validate_cash:1, validate_position:1, enable_auto_planed_order:0
INFO:MAIN:login_account(bktfut)
INFO:MAIN:> add_strategy setting:{'strategy_name': 'strategy', 'account_id': 'bktfut'}
INFO:MAIN:init all strategy account_id:...
INFO:MAIN:stop all strategy account_id:...


[2024-12-03 16:03:42] [info     ] backtest done, raw_perf_ds:dai.DataSource("_03731cf3f99f4fc189444f54391f8dbf")


[2024-12-03 16:03:42] [info     ] bigtrader.v34 运行完成 [12.405s].


,date,trading_days,portfolio_value,starting_value,ending_value,starting_cash,ending_cash,returns,algorithm_period_return,benchmark_period_return,...,today_sell_balance,commission,capital_changed,orders,transactions,positions,POS_FAC,TRA_FAC,LOG,benchmark_returns
2024-11-01,2024-11-01,1,999087.519499,0.0,0.0,1.000008e+06,991358.519499,-0.000921,-0.000921,0.000000,...,204917.0,570.59,0.0,"[{'symbol': 'AP8888.CZC', 'id': '1', 'dt': 202...","[{'symbol': 'AP8888.CZC', 'dt': 2024-11-01 09:...","[{'symbol': 'AP8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[09:28:00][id:...",0.000000
2024-11-04,2024-11-04,2,997553.549499,0.0,0.0,9.913585e+05,997553.549499,-0.001535,-0.002455,0.014072,...,352773.5,653.97,0.0,"[{'symbol': 'AP8888.CZC', 'id': '95', 'dt': 20...","[{'symbol': 'jm8888.DCE', 'dt': 2024-11-01 21:...","[{'symbol': 'AP8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",0.014072
2024-11-05,2024-11-05,3,996569.809499,0.0,0.0,9.975535e+05,996569.809499,-0.000986,-0.003438,0.039730,...,256869.0,543.74,0.0,"[{'symbol': 'PR8888.CZC', 'id': '230', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2024-11-04 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",0.025302
2024-11-06,2024-11-06,4,995953.249499,0.0,0.0,9.965698e+05,995953.249499,-0.000619,-0.004055,0.034515,...,323159.0,646.56,0.0,"[{'symbol': 'PR8888.CZC', 'id': '338', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2024-11-05 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:03:00][id:...",-0.005016
2024-11-07,2024-11-07,5,995122.989499,0.0,0.0,9.959532e+05,979042.989499,-0.000834,-0.004885,0.065726,...,283397.0,770.26,0.0,"[{'symbol': 'jm8888.DCE', 'id': '470', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2024-11-06 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:04:00][id:...",0.030170
2024-11-08,2024-11-08,6,992631.309499,0.0,0.0,9.790430e+05,992631.309499,-0.002504,-0.007377,0.055019,...,290356.0,651.68,0.0,"[{'symbol': 'jm8888.DCE', 'id': '600', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2024-11-07 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",-0.010046
2024-11-11,2024-11-11,7,991276.739499,0.0,0.0,9.926313e+05,975742.739499,-0.001365,-0.008731,0.061982,...,240253.5,514.57,0.0,"[{'symbol': 'PR8888.CZC', 'id': '727', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2024-11-08 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:16:00][id:...",0.006599
2024-11-12,2024-11-12,8,989891.619499,0.0,0.0,9.757427e+05,989891.619499,-0.001397,-0.010116,0.050314,...,312362.5,655.12,0.0,"[{'symbol': 'jm8888.DCE', 'id': '825', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2024-11-11 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",-0.010987
2024-11-13,2024-11-13,9,989462.799499,0.0,0.0,9.898916e+05,989462.799499,-0.000433,-0.010545,0.056779,...,223586.0,488.82,0.0,"[{'symbol': 'PR8888.CZC', 'id': '958', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2024-11-12 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:02:00][id:...",0.006155
2024-11-14,2024-11-14,10,988262.719499,0.0,0.0,9.894628e+05,988262.719499,-0.001213,-0.011745,0.038457,...,371701.5,770.08,0.0,"[{'symbol': 'PR8888.CZC', 'id': '1054', 'dt': ...","[{'symbol': 'PR8888.CZC', 'dt': 2024-11-13 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",-0.017337
